# CENG403 - Spring 2024 - THE3
In this THE, you will first implement convolution & pooling operations of a CNN and their backpropagation(Task 1). Then, you will experience how CNNs can be implemented in PyTorch(Task2). Finally, you will finetune a ResNet18 (prerained on ImageNet) on CIFAR10 (Task3).

Note that these Colab files are provided read-only and you should copy them to your Google Drives/Colab folders to be able to save your changes.

Once you are done working on your files, without clearing the outputs, save them on your computer and zip them into a file THE3.zip without placing them into a directory. Upload this ZIP file on the ODTUClass page of the course under assignment "THE3".
# Task 1: Naive Convolution and Pooling
In this task, you will implement convolution and pooling operations in a naive way without using any matrix-vector multiplication.

*Disclaimer: Some components are adapted or taken from [CS231n](https://cs231n.github.io/) materials.*

## 1.1 Import the Modules

As usual, let us import the modules that we will use.

In [63]:
%pip install matplotlib
import matplotlib.pyplot as plt # For plotting
import numpy as np              # NumPy, for working with arrays/tensors
import os                       # Built-in library for filesystem access etc.
import pickle                   # For (re)storing Python objects into (from) files
import time                     # For measuring time
import random                   # Python's random library

%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

Note: you may need to restart the kernel to use updated packages.


## 1.2 Naive Implementation of Convolution

Here, we will implement 2D convolution using only for/while loops; in other words, **NO** vector/matrix multiplications.

### 1.2.1 Convolution Feedforward




In [64]:
def conv_forward_naive(x, w, b, stride, padding):
    """
    The input consists of N samples, each with C channels, height H and
    width W. We convolve each input with F different filters, where each filter
    spans all C channels and has height FH and width FW.

    Input:
    - x: Input data of shape (N, C, H, W)
    - w: Filter weights of shape (F, C, FH, FW)
    - b: Biases, of shape (F,)
    - stride: The number of pixels between adjacent receptive fields in the
        horizontal and vertical directions.
    - padding: The number of pixels that will be used to zero-pad the input.

    During padding, 'padding'-many zeros should be placed symmetrically (i.e equally on both sides)
    along the height and width axes of the input. Be careful not to modfiy the original
    input x directly.

    Returns a tuple of:
    - out: Output data, of shape (N, F, H', W') where H' and W' are given by
      H' = 1 + (H + 2 * padding - FH) / stride
      W' = 1 + (W + 2 * padding - FW) / stride
    - cache: (x_pad, w, b, stride, padding)
    """

    # Layer output:
    out = None

    # Get values for the dimensions:
    N, C, H, W = x.shape
    F, C, FH, FW = w.shape

    # Just to make sure
    if (H - FH + 2 * padding) % stride != 0:
      return ValueError("Filters do not align horizontally")
    if (W - FW + 2 * padding) % stride != 0:
      return ValueError("Filters do not align vertically")
    
    


    ###########################################################################
    # TODO: Implement the convolutional forward pass. You cannot use          #
    # vector or matrix multiplications here. Be careful about stride.         #
    # Hints:                                                                  #
    #  (1) You can use the function np.pad for padding.                       #
    #  (2) You will have such a nested loop structure here:                   #
    #    - loop over N samples                                                #
    #      - loop over F filters                                              #
    #        - loop over rows of x_pad                                        #
    #          - loop over cols of x_pad                                      #
    #            - loop over kernel rows                                      #
    #              - loop over kernel cols                                    #
    #                - loop over channels                                     #
    ###########################################################################
    out_H = 1 + (H + 2 * padding - FH) // stride # +Calculate the output height
    out_W = 1 + (W + 2 * padding - FW) // stride # +Calculate the output width
    
    x_pad = np.pad(x, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant', constant_values=0) # +Pad the input

    out = np.zeros((N, F, out_H, out_W)) # +Initialize the output
    
    for n in range(N):
      for f in range(F):
        for i in range(out_H): # +Iterate over the output height
          for j in range(out_W): # +Iterate over the output width
            h_start = i * stride
            h_end = h_start + FH # +Calculate the height start and end of the "current" slice
            w_start = j * stride
            w_end = w_start + FW
            for k_h in range(FH):
              for k_w in range(FW):
                for c in range(C):
                  out[n, f, i, j] += x_pad[n, c, h_start + k_h, w_start + k_w] * w[f, c, k_h, k_w] # +Convolve the filter with the input
            out[n, f, i, j] += b[f]  # Add bias term !!!! 
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    cache = (x_pad, w, b, stride, padding)
    return out, cache

### 1.2.2 Convolution Feedforward Check

Let us quickly check your implementation with some known values. Your output should differ by $\sim 10^{-8}$.

In [65]:

x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3)

stride = 2
padding = 1
out, _ = conv_forward_naive(x, w, b, stride, padding)
correct_out = np.array([[[[-0.08759809, -0.10987781],
                           [-0.18387192, -0.2109216 ]],
                          [[ 0.21027089,  0.21661097],
                           [ 0.22847626,  0.23004637]],
                          [[ 0.50813986,  0.54309974],
                           [ 0.64082444,  0.67101435]]],
                         [[[-0.98053589, -1.03143541],
                           [-1.19128892, -1.24695841]],
                          [[ 0.69108355,  0.66880383],
                           [ 0.59480972,  0.56776003]],
                          [[ 2.36270298,  2.36904306],
                           [ 2.38090835,  2.38247847]]]])

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Compare your output to ours; difference should be around e-8
print('Testing conv_forward_naive')
print('difference: ', rel_error(out, correct_out))

Testing conv_forward_naive
difference:  2.212147649671884e-08


### 1.2.3 Convolution Gradient

Now, let us implement the gradient. Again, you are **NOT** allowed to use any vector/matrix multiplication here.

In [66]:
def conv_backward_naive(dout, cache):
    """
    A naive implementation of the backward pass for a convolutional layer.

    Inputs:
    - dout: Upstream derivatives.
    - cache: A tuple of (x, w, b, stride, padding) as in conv_forward_naive

    Returns a tuple of:
    - dx: Gradient with respect to x.
    - dw: Gradient with respect to w
    - db: Gradient with respect to b
    """
    dx, dw, db = None, None, None
    x, w, b, stride, padding = cache

    N, C, H, W = x.shape
    F, C, FH, FW = w.shape
    N, F, out_H, out_W = dout.shape
    
    dx = np.zeros_like(x)
    dw = np.zeros_like(w)
    db = np.zeros_like(b)
    
    x_pad = np.pad(x, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant', constant_values=0)
    # dx_pad = np.pad(dx, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant', constant_values=0)
    dx_pad = np.zeros_like(x_pad)
    
    
    db = np.sum(dout, axis=(0, 2, 3))
    
    for n in range(N):
      for f in range(F):
        for i in range(out_H):
          for j in range(out_W):
            h_start = i * stride
            h_end = h_start + FH
            w_start = j * stride
            w_end = w_start + FW
            
            for k_h in range(FH):
              for k_w in range(FW):
                for c in range(C):
                  dw[f, c, k_h, k_w] += x_pad[n, c, h_start + k_h, w_start + k_w] * dout[n, f, i, j] #+ Calculate the gradient with respect to w
            for k_h in range(FH):
              for k_w in range(FW):
                for c in range(C):
                  dx_pad[n, c, h_start + k_h, w_start + k_w] += w[f, c, k_h, k_w] * dout[n, f, i, j] #+ Calculate the gradient with respect to x
    dx = dx_pad[:, :, padding:-padding, padding:-padding]
        
    # if padding > 0:
    #     dx = dx_pad[:, :, padding:-padding, padding:-padding]
    # else:
    #     dx = dx_pad
        
    print("***** DIMENSIONS *****")
    print('x_pad shape:', x_pad.shape)
    print('dx_pad shape:', dx_pad.shape)
    print('dx shape:', dx.shape)
    print('dw shape:', dw.shape)
    print('db shape:', db.shape)
    ###########################################################################
    # TODO: Implement the convolutional backward pass.                        #
    ###########################################################################

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return dx, dw, db

### 1.2.4 Convolution Gradient Check

Let us check the gradient. You should see differences lower than $10^{-9}$.

In [67]:
def eval_numerical_gradient_array(f, x, df, h=1e-5):
    """
    Evaluate a numeric gradient for a function that accepts a numpy
    array and returns a numpy array.
    """
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index

        oldval = x[ix]
        x[ix] = oldval + h
        pos = f(x).copy()
        x[ix] = oldval - h
        neg = f(x).copy()
        x[ix] = oldval

        grad[ix] = np.sum((pos - neg) * df) / (2 * h)
        it.iternext()
    return grad

np.random.seed(231)
x = np.random.randn(4, 3, 5, 5)
w = np.random.randn(2, 3, 3, 3)
b = np.random.randn(2,)
dout = np.random.randn(4, 2, 5, 5)
stride = 1
pad = 1

dx_num = eval_numerical_gradient_array(lambda x: conv_forward_naive(x, w, b, stride, pad)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv_forward_naive(x, w, b, stride, pad)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: conv_forward_naive(x, w, b, stride, pad)[0], b, dout)

out, cache = conv_forward_naive(x, w, b, stride, pad)
dx, dw, db = conv_backward_naive(dout, cache)
print("***** GRADIENTS *****")
print(dx_num)
# Your errors should be around e-8 or less.
print('Testing conv_backward_naive function')
print('dx error: ', rel_error(dx, dx_num))
print('dw error: ', rel_error(dw, dw_num))
print('db error: ', rel_error(db, db_num))

***** DIMENSIONS *****
x_pad shape: (4, 3, 9, 9)
dx_pad shape: (4, 3, 9, 9)
dx shape: (4, 3, 7, 7)
dw shape: (2, 3, 3, 3)
db shape: (2,)
***** GRADIENTS *****
[[[[ 2.02022648e+00  1.31557244e+00 -3.15087854e+00  3.23892197e+00
    -4.01214844e+00]
   [ 1.71832340e+00 -7.09240980e-01 -5.32421591e+00  7.82298212e-01
    -2.18995738e+00]
   [-8.29200893e-01  2.82134457e+00 -1.25541821e+00 -1.13170679e+00
    -3.55839346e+00]
   [-3.35783037e-01 -2.76913948e+00  4.25723911e+00 -8.11824513e-01
     2.70901018e+00]
   [ 8.47052635e-02  4.30588591e-01  2.95022069e+00 -2.92763735e+00
    -9.38719074e-01]]

  [[ 3.36982195e-01  7.66793750e+00  1.57619395e+00  5.02203786e+00
    -1.69753665e+00]
   [-1.93329123e+00 -9.86659460e+00  3.53426045e+00  7.35637014e-01
    -3.43491944e+00]
   [-5.79144643e-01  6.55462097e-01  1.89050154e+00 -6.40445092e-01
    -2.87212774e+00]
   [-7.13485097e+00  8.58935000e+00 -3.51576934e+00  4.39354129e+00
    -4.79888306e-02]
   [ 7.41454674e-02  5.09290866e+00 -3

ValueError: operands could not be broadcast together with shapes (4,3,7,7) (4,3,5,5) 

## 1.3 Pooling

Now, we implement pooling, max-pooling to be specific. Again, **no** use of vector/matrix multiplications.

### 1.3.1 Pooling Feedforward

In [ ]:
def max_pool_forward_naive(x, stride, PH, PW):
    """
    A naive implementation of the forward pass for a max-pooling layer.
    Inputs:
    - x: Input data, of shape (N, C, H, W)
    - stride: The distance between adjacent pooling regions
    - PH: The height of each pooling region
    - PW: The width of each pooling region

    Returns a tuple of:
    - out: Output data, of shape (N, C, H', W') where H' and W' are given by
      H' = 1 + (H - pool_height) / stride
      W' = 1 + (W - pool_width) / stride
    - cache: (x, pool_param)
    """
    out = None
    ###########################################################################
    # TODO: Implement the max-pooling forward pass                            #
    ###########################################################################

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    cache = (x, stride, PH, PW)
    return out, cache

### 1.3.2 Pooling Feedforward Check

You should observe ~$10^{-8}$ difference.

In [ ]:
x_shape = (2, 3, 4, 4)
x = np.linspace(-0.3, 0.4, num=np.prod(x_shape)).reshape(x_shape)
stride, PH, PW = (2, 2, 2)

out, _ = max_pool_forward_naive(x, stride, PH, PW)

correct_out = np.array([[[[-0.26315789, -0.24842105],
                          [-0.20421053, -0.18947368]],
                         [[-0.14526316, -0.13052632],
                          [-0.08631579, -0.07157895]],
                         [[-0.02736842, -0.01263158],
                          [ 0.03157895,  0.04631579]]],
                        [[[ 0.09052632,  0.10526316],
                          [ 0.14947368,  0.16421053]],
                         [[ 0.20842105,  0.22315789],
                          [ 0.26736842,  0.28210526]],
                         [[ 0.32631579,  0.34105263],
                          [ 0.38526316,  0.4       ]]]])

# Compare your output with ours. Difference should be on the order of e-8.
print('Testing max_pool_forward_naive function:')
print('difference: ', rel_error(out, correct_out))

### 1.3.3 Pooling Gradient

Again, **no** use of vector/matrix multiplications.

In [ ]:
def max_pool_backward_naive(dout, cache):
    """
    A naive implementation of the backward pass for a max-pooling layer.
    Inputs:
    - dout: Upstream derivatives
    - cache: A tuple of (x, stride, PH, PW) as in the forward pass.
    Returns:
    - dx: Gradient with respect to x
    """
    dx = None
    ###########################################################################
    # TODO: Implement the max-pooling backward pass                           #
    ###########################################################################

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return dx

### 1.3.4 Pooling Gradient Check

You should observe ~$10^{-12}$ difference.

In [ ]:
np.random.seed(231)
x = np.random.randn(3, 2, 8, 8)
dout = np.random.randn(3, 2, 4, 4)
stride, PH, PW = (2, 2, 2)

dx_num = eval_numerical_gradient_array(lambda x: max_pool_forward_naive(x, stride, PH, PW)[0], x, dout)

out, cache = max_pool_forward_naive(x, stride, PH, PW)
dx = max_pool_backward_naive(dout, cache)

# Your error should be on the order of e-12
print('Testing max_pool_backward_naive function:')
print('dx error: ', rel_error(dx, dx_num))